In [1]:
!pip install -U -q lightautoml

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.19.1 requires aiohttp, which is not installed.
tensorflow-decision-forests 1.8.1 requires wurlitzer, which is not installed.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.2.1 which is incompatible.
dask-expr 1.1.1 requires pandas>=2, but you have pandas 1.5.3 which is incompatible.
featuretools 1.31.0 requires pandas>=2.0.0, but you have pandas 1.5.3 which is incompatible.
fitter 1.7.0 requires joblib<2.0.0,>=1.3.1, but you have joblib 1.2.0 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
mizani 0.11.4 requires pandas>=2.1.0, but you have pandas 1.5.3 which is incompatible.
momepy 0.7.0 requires shapely

# Configs and Imports

In [2]:
from lightautoml.automl.presets.tabular_presets import TabularAutoML
from lightautoml.tasks import Task
import pandas as pd
import numpy as np
import torch
import pickle

In [3]:
TARGET_NAME = 'Target'
TEST_SIZE = 0.1
N_THREADS = 4
TIMEOUT = 3600 * 10
N_FOLDS = 5
SEED = 27

In [4]:
np.random.seed(SEED)
torch.set_num_threads(N_THREADS)

# Loading and Preprocessing the Datasets

In [5]:
train = pd.read_csv('/kaggle/input/playground-series-s4e6/train.csv', index_col='id')
test = pd.read_csv('/kaggle/input/playground-series-s4e6/test.csv', index_col='id')

train.shape, test.shape

((76518, 37), (51012, 36))

In [6]:
original = pd.read_csv('/kaggle/input/students-dropout-and-academic-success/data.csv', delimiter=';')
original = original.rename(columns={'Daytime/evening attendance\t': 'Daytime/evening attendance'})

train = pd.concat([train, original], axis=0, ignore_index=True)

train.shape

(80942, 37)

In [7]:
# Reference: https://www.kaggle.com/code/ambrosm/pss4e6-eda-which-makes-sense
cat_features = [
    'Marital status', 
    'Application mode', 
    'Course',
    'Previous qualification', 
    'Nacionality',
    "Mother's qualification",
    "Father's qualification",
    "Mother's occupation",
    "Father's occupation"
]
for feature in cat_features:
    for df in [train, test]:
        df[feature] = df[feature].astype('category').cat.codes

# Training

In [8]:
task = Task(
    name='multiclass',
    greater_is_better=True
)

In [9]:
automl = TabularAutoML(
    task=task,
    timeout=TIMEOUT,
    cpu_limit=N_THREADS,
    reader_params={
        'n_jobs': N_THREADS,
        'cv':  N_FOLDS,
        'random_state': SEED
    },
    tuning_params = {'max_tuning_time': 3600},
)

oof_preds = automl.fit_predict(
    train,
    roles={'target': TARGET_NAME},
    verbose=1
)

[09:21:44] Stdout logging level is INFO.
[09:21:44] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[09:21:44] Task: multiclass

[09:21:44] Start automl preset with listed constraints:
[09:21:44] - time: 36000.00 seconds
[09:21:44] - CPU: 4 cores
[09:21:44] - memory: 16 GB

[09:21:44] Train data shape: (80942, 37)

[09:21:55] Layer 1 train process start. Time left 35989.27 secs
[09:22:00] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[09:22:16] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = 0.5363263111594837
[09:22:16] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[09:22:16] Time left 35967.92 secs

[09:22:37] Selector_LightGBM fitting and predicting completed
[09:22:43] Start fitting Lvl_0_Pipe_1_Mod_0_LightGBM ...
[09:26:51] Fitting Lvl_0_Pipe_1_Mod_0_LightGBM finished. score = 0.43651802483124796
[09:26:51] Lvl_0_Pipe_1_Mod_0_LightGBM fitting and predicting completed
[09:26:51] Start hyperparameters optimizati

In [10]:
with open('lightautoml_oof_preds.pkl', 'wb') as f:
    pickle.dump(oof_preds.data, f)

# Post-Processing and Inference

In [11]:
preds = automl.predict(test).data

In [12]:
class_mapping = automl.reader.class_mapping
class_mapping = {v: k for k, v in class_mapping.items()}
class_mapping

{0: 'Graduate', 1: 'Dropout', 2: 'Enrolled'}

In [13]:
submission = pd.read_csv('/kaggle/input/playground-series-s4e6/sample_submission.csv', index_col='id')

submission[TARGET_NAME] = preds.argmax(axis=1)
submission[TARGET_NAME] = submission[TARGET_NAME].map(class_mapping)

submission.to_csv('submission.csv')
submission.head()

,Target
id,
76518,Dropout
76519,Graduate
76520,Graduate
76521,Graduate
76522,Enrolled
